## 상품정보 데이터

### json데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

import re
import warnings
import os
warnings.filterwarnings(action='ignore')
#pd.set_option('옵션 이름', 수정할 값)
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [ ]:
items = pd.read_json('./items.json')
items.head()



In [ ]:
items.info()

### 상품명 전처리

In [ ]:
def item_name(row):
    return row.split('\n')[0] # 줄바꿈이전 문자열만 남기기

items['name'] = items['name'].apply(item_name)
items.head()

### 상품명 중복확인

In [ ]:
dup_idx = items.loc[items['name'].duplicated()==True].index
items.drop(dup_idx, axis=0, inplace=True)
items['name'].duplicated().sum()


In [ ]:
# 인덱스 재설정
items.reset_index(drop=True, inplace=True)

In [ ]:
items.info()

### Item_id 부여

In [ ]:
items['item_id'] = [(5-len((str(i)))) * '0' + str(i) for i in list(items.index)]
items.head()

In [ ]:
items.tail()

In [ ]:
# items.to_json('../Load/items_prc.json', orient='records', indent=2, force_ascii=False) # 객체기준 레코드, 들여쓰기 적용, 한글깨짐 해결

In [ ]:
# # 리뷰 크롤링용 데이터 저장
# items[['item_id', 'link']].to_json('../Extract/items_link.json', orient='records', indent=2, force_ascii=False)

# review data

In [ ]:
review = pd.read_json('./reviews.json')

# 불필요한 컬럼 삭제
review = review[['품번', '아이디', '별점', '키', '몸무게', '평소사이즈', '타입', '리뷰', '사이즈', '컬러', '색상', '색상-사이즈', '타입2', '데님']]
review.info()


In [ ]:
review.head()

### 카디널리티 확인

In [ ]:
for col in ['별점', '키', '몸무게', '평소사이즈']:
    print(col)
    print(review[col].unique())
    print('\n')

* 핫핑 사이즈 기준

<img src='./imgs/사이즈통일.png'>

In [ ]:
# 평소 사이즈 수정
key1 = [str(i) for i in range(44, 100, 11)] +['100']
key2 = ['S', 'M', 'L', 'XL', 'XXL', 'XXXL']

def user_size(row):
    for i in range(1,6):
        if row == key1[i]:
            return str(i)
        elif row == key2[i]:
            return str(i)
        
review['평소사이즈'] = review['평소사이즈'].apply(user_size)
review['평소사이즈'].unique()

### ['사이즈', '컬러', '색상', '타입', '타입2', '색상-사이즈', '데님'] 컬럼 통일 필요 = **['사이즈: str', '색상: 한글']으로 통일**


<img src='./imgs/타입_해당컬럼.png' width=500 height=500/>

#### 각 컬럼별 카디널리티 줄이고 컬럼 통일

In [ ]:
# 카디널리티 확인
cols = ['사이즈', '컬러', '색상', '타입', '타입2', '색상-사이즈', '데님']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

In [ ]:
# 사이즈 컬럼
def size(row):
    if row in ['1', '2', '3', '4', 'F', '블랙(black)-2', '블랙 1']:
        return row
    elif row in ['1size', '미니-1', '1사이즈']:
        return re.sub('[^0-9]', '', row)
    elif row=='free':
        return 'F'
review['사이즈'] = review['사이즈'].apply(size)

review.loc[review['사이즈']=='블랙 1'] = '블랙-1'

# 컬러, 색상 컬럼
def color(row):
    if type(row) != float:
        return row
    
review['컬러'] = review['컬러'].apply(color)
review['색상'] = review['색상'].apply(color)

# 타입 컬럼
def type_prc(row):
    if not row:
        pass
    elif '. ' in row: 
        return row.split('-')[-1]
    elif '[예약판매]' in row:
        row = row.replace('[예약판매]', '')
        return row
    elif '(반팔)' in row:
        row = row.replace('(반팔)', '')
        return row
    elif '_' in row:
        return row.split('_')[-1]
    elif '.' in row:
        return None
    elif type(row)==str:
        return row

review['타입'] = review['타입'].apply(type_prc)
    
# 데님, 색상-사이즈 컬럼
def denim_color(row):
    if type(row) == str:
        if '예약' in row:
            return row.split(' ')[0]
        else:
            return row

review['데님'] = review['데님'].apply(denim_color)
review['색상-사이즈'] = review['색상-사이즈'].apply(denim_color)
review['타입2'] = review['타입2'].apply(denim_color)

전: <img src='./imgs/카디널리티_전.png'> 후: <img src='./imgs/카디널리티_후.png'>

#### ['색상', '사이즈'] 컬럼으로 통일

In [ ]:
# 카디널리티 확인
cols = ['사이즈', '컬러', '색상', '타입', '타입2', '색상-사이즈', '데님']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

In [ ]:
# 색상-사이즈 형태인 경우
    ## str.split()로 분리, 문자열 정규화로 데이터 수정
# 색상 데이터만 있는 경우
    ## 문자열 정규화만 수행
def col_merge(df, cols):
    def normalize(row):
        row[0] = re.sub('[^가-힣]','',row[0]) # 한글만 남기기
        return row

    # 사이즈 컬럼 별도 수행
    idx = df.loc[(review['사이즈']=='블랙-1') | (review['사이즈']=='블랙(black)-2')].index
    for i in idx:
        df.loc[i, ['색상', '사이즈']] = df.iloc[i]['사이즈'].split('-')
    for col in cols:
        idx = df.loc[df[col].notnull()==True].index
        for i in idx:
            row = df.iloc[i][col].split('-')
            if len(row)==2:
                df.loc[i, ['색상', '사이즈']] = normalize(row)
            else:
                df.loc[i, ['색상']] = normalize(row)[0]
    return df
cols = ['컬러', '타입', '타입2', '색상-사이즈', '데님', '색상']
review = col_merge(review, cols)
review.head()

* 나머지 컬럼은 삭제

In [ ]:
# 통합 끝난 컬럼 제거 (색상, 사이즈)
review.drop(columns=cols[:-1], inplace=True)

* 카디널리티 확인

In [ ]:
def last_prc(col, string):
    idx = review.loc[review[col]==string].index
    for i in idx:
        review.iloc[i][col] = None

In [ ]:
['미니기장블랙', '롱블랙']
['롱', '미니']

In [85]:
cols = ['사이즈', '색상']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

사이즈
['2' '3' None '1' '4' 'F']


색상
['블랙' '소라' None '썸머블랙' '네이비' '브라운' '진베이지' '블루' '딥네이비' '화이트' '카키' '옐로우'
 '크림' '연청' '데님' '미니기장블랙' '오트베이지' '롱블랙' '진청' '아이보리' '퍼플' '레드' '그레이'
 '오렌지브라운' '오트밀' '그린' '라이트코랄' '오렌지' '핑크' '연두' '연핑크' '머스타드' '엘로우' '미니' '롱'
 '베이지' '후드' '카라' '코랄' '민트' '도트블랙' '긴팔버전' '연베이지' '중청' '라이트블루' '스카이블루' '스킨'
 '크림아이보리' '선셋핑크' '스카이' '모카베이지' '소프트블루' '연카키' '차콜' '옐로우베이지' '옐로우그린' '와인'
 '뮤트블루' '더스트블루' '올리브그린' '크림베이지' '카키브라운' '올리브' '네이' '더스트핑크' '에쉬베이지' '포그블루'
 '레몬옐로우' '라이트오렌지' '딥블루' '진네이비' '블루그레이' '모카' '롱진청' '애쉬민트' '연블루' '라이트브라운'
 '다홍' '멜란지그레이' '라벤더' '코발트블루' '카멜베이지' '페일옐로우' '딥핑크' '썸머블루' '진파랑' '망고옐로우'
 '하트버튼' '퍼프라운드' '라운드' '브이넥' '브이' '라이트퍼플' '포그카키' '뮤트핑크' '연그레이' '백메란지' '라임'
 '피콕블루' '브릭' '연보라' '라이트옐로우' '다크그레이' '핫핑크' '네온' '레몬' '옐로네온']




### '별점' 컬럼 핫핑 직원 데이터는 제외

In [ ]:
staffs = set()
p = re.compile('[ㄱ-힣]')
for name in review['아이디'].unique():
    r = p.search(name)
    if r:
        staffs.add(name)
          

In [ ]:
staffs